# Google Quickstart

In this quickstart, you will learn to build and evaluate a RAG (Retrieval-Augmented Generation) application using Google's powerful Gemini models, with a focus on getting started quickly in Google AI Studio.

Building and evaluating RAG applications with Google Gemini gives you access to a suite of state-of-the-art, multi-modal models.

In this example, we will use Gemini models for embedding, generation, and as the LLM judge to power TruLens feedback functions. You can start building immediately in Google AI Studio with just a Google account.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)]()

In [ ]:
# !pip install trulens trulens-providers-google chromadb

In [ ]:
import os

os.environ["GOOGLE_API_KEY"] = "sk-..."

## Get Data

In this case, we'll just initialize some simple text in the notebook.

In [ ]:
university_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

In [ ]:
import chromadb

chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(name="Universities")

In [ ]:
from google import genai


google_client = genai.Client()

document_embeddings = google_client.models.embed_content(model="gemini-embedding-001",
        contents= university_info).embeddings

In [ ]:
vector_store.add(
    "uni_info", documents=university_info, embeddings=document_embeddings[0].values
)

## Build RAG from scratch

Build a custom RAG from scratch, and add TruLens custom instrumentation.

In [ ]:
from trulens.apps.app import instrument
from trulens.core import TruSession


session = TruSession()
session.reset_database()

In [ ]:
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(
            query_embeddings=google_client.models.embed_content(
                model="gemini-embedding-001", contents=query
            )
            .embeddings[0]
            .values,
            n_results=2,
        )
        return results["documents"]

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        prompt = f"""
         We have provided context information below. 
            {context_str}
            Given this information, please answer the question: {query}
        """
        resp = google_client.models.generate_content(
            model="gemini-2.5-flash", contents=prompt
        )

        return resp.text if resp else ""

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion


rag = RAG_from_scratch()

## Set up feedback functions.

Here we'll use groundedness, answer relevance and context relevance to detect hallucination.

In [ ]:
import numpy as np
from trulens.core import Feedback
from trulens.core import Select
from trulens.providers.google import Google

provider = Google()

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)
                      
f_coherence = Feedback(
    provider.coherence_with_cot_reasons, name="coherence"
).on_output()

## Construct the app

Wrap the custom RAG with `TruApp`, add list of feedbacks for eval

In [ ]:
from trulens.apps.app import TruApp

tru_rag = TruApp(
    rag,
    app_name="RAG",
    app_version="v1",
    feedbacks=[
        f_groundedness,
        f_answer_relevance,
        f_context_relevance,
        f_coherence,
    ],
)

## Run the app

Use `tru_rag` as a context manager for the custom RAG-from-scratch app.

In [ ]:
with tru_rag as recording:
    resp = rag.query("When is University of Washington founded?")

In [ ]:
resp

In [ ]:
session.get_leaderboard(app_ids=[])

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(session)